# Trading Places
In this homework, you will use the BACI bilateral trade flows dataset to analyze trade flows for the world and, more specifically, for the country of __Uruguay__. You will also do a rudimentary analysis of what is referred to as the "gravity equation" from international trade theory.

1. Read in every year of available data (2012 to 2020) using Dask __(1 point)__
2. Descriptive Statistics: __(3 points)__
    
    a. In the year 2012, who were the top 10 countries with the most trading partners? What about the bottom 10?
    
    b. Using the designation [here]( https://www.foreign-trade.com/reference/hscode.htm), describe the trade volume of the whole dataset in terms of value and list the five highest value sectors in the entire sample period.

    c. Calculate the top 10 goods with the highest trade volume in the entire dataset in terms of:
    
        i. Value
    
        ii. Quantity
3. Country statistics: __(2 points)__

    a. Calculate the top 10 exports (in terms of value) of Uruguary in 2012.
    
    b. Using the aggregated categories, find the product category with the highest annual average value from 2012 to 2020 in Uruguary and plot imports and export of this category from 2012 to 2020.



4. The Gravity Equation Relationship __(6 points)__

    a. (2 point) Using the country shapefile, calculate the distance in kilometers between the centroid of Uruguay and all of its trading partners using haversine distance or Euclidean distance (depending on what projection you are using).
    
    b. (1 point) Create a scatterplot of distance and trade flow volume (quantity and value) in logarithm form.
    
    c. (1 point) Calculate the correlations between trade volume and distance for both value and quantity. Interpret the correlations for each of these factors.
    
    d. (2 point) Using the designation you created in 2.d., for which sectors are trade flows (quantity) most strongly related to distance for this country?
    
5. Estimation __(5 points)__. For all the next questions, use the Uruguay trade flows (both exports and imports).
    
    a. (1 point) Create a function called "SSE_1p" that does the following given a scalar input $\beta$:
        1. subtracts the logarithm of quantity flow from the logarithm of distance times $\beta$.
$$ e = log(F_{ij}) - \beta log(d_{ij}) $$
        2. Squares the error.
        3. Returns the sum of the squared error. 
    b. (1 point) Plot the function from -1 to +3 with steps of .001, and then find the minimum of the function with the numpy function "argmin." Is it what you expected?
    
    c. (1 point) Time the above function, which is a grid search done sequentially. Then, use `dask` or `multiprocess` to  parallelize the function above and time it. What are the differences? 
    
    d. __EXTRA CREDIT (4 points)__ 
        i. Create another function called "SSE_2p" does the same as "SSE_1p" but instead takes a __vector__ as an argument:
        
 $$ e = log(F_{ij}) - \beta[0] - \beta[1] log(d_{ij})$$
 
        ii. Do a grid search with the first parameter range as 1500 to 2500 in steps of 10, and the second parameter range as -1 to +3 in steps of .005 as before. Do it using either `dask` or `multiprocessing`. What were your results? Comment on how or why the parameter on $log(d_{ij})$ is different than what you found in part 4b.
                NOTE: if it hits the lower bound, this means you should try a different parameter range to maximize over.
     
     e. (1 point) Now use the "econtools" package "reg" function to estimate the equation from part a but instead with a constant term. How much did the distance coefficient change?
     
     f. (1 point) Finally, merge in the GDP numbers from the World Bank and do the regression again, except this time adding the log of each country's GDP as covariates. How did the coefficient on distance change?
     
5. An analyst asks you "for which commodity sectors is distance most important in terms of exporting?" Describe what your economic intuition would say about the answer, and then also map out an analysis strategy for answering their question. As usual, describe: __(3 points)__
    - What data you would need.
    - How you would manipulate the data.
    - What analysis you would run.

In [1]:
import glob
import pandas as pd
from econtools.metrics import reg
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, progress
import multiprocess as mp
import matplotlib.pyplot as plt
import json
import dask
import geopandas as gp
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from multiprocessing.pool import ThreadPool, Pool
from dask import delayed
from dask import compute

def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = row
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r